In [1]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
min_mem_size=20 
run_time=222

In [3]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
# min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

20


In [4]:
h2o.init(strict_version_check=False,max_mem_size=min_mem_size)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 hour 4 mins
H2O cluster version:,3.12.0.1
H2O cluster version age:,"1 year, 9 months and 4 days !!!"
H2O cluster name:,H2O_from_python_prabh_xeria3
H2O cluster total nodes:,1
H2O cluster free memory:,3.400 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [5]:
data_path = "data/Amazon_review.csv"

# Load data into H2O
reviews = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
reviews.shape

(48181, 21)

In [7]:
reviews.describe()

Rows:48181
Cols:21




,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
type,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,enum,string,enum,enum,string,enum
mins,,,,,,,,,,,,,0.0,0.0,0.0,,NaN,,,NaN,
mean,,,,,,,,,,,,,18689.87867091794,1.4077145669878584,4.565938989855542,,NaN,,,0.0,
maxs,,,,,,,,,,,,,111372787.0,814.0,19.0,,NaN,,,NaN,
sigma,,,,,,,,,,,,,1442754.3578338078,12.529322130002884,0.7621248960567992,,NaN,,,-0.0,
zeros,,,,,,,,,,,,,5858,24386,20,,0,,,0,
missing,0,20137,13393,6777,13505,13528,13408,19490,17441,19697,41820,20079,42222,14660,20284,13527,13537,19683,47374,41844,20080
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Magenta",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Tablets,Tablets,Computers & Tablets","841667104676,amazon/53004484,amazon/b01ahb9cn2,0841667104676,allnewfirehd8tablet8hddisplaywifi16gbincludesspecialoffersmagenta/5620406,allnewfirehd8tablet8hddisplaywifi16gbincludesspecialoffersmagenta/b01ahb9cn2",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",,TRUE,nan,0.0,5.0,"http://reviews.bestbuy.com/3545/5620406/reviews.htm?format=embedded&page=200,http://reviews.bestbuy.com/3545/5620406/reviews.htm?format=embedded&page=166",This product so far has not disappointed. My children love to use it and I like the ability to monitor control what content they see with ease.,Kindle,,,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Magenta",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Tablets,Tablets,Computers & Tablets","841667104676,amazon/53004484,amazon/b01ahb9cn2,0841667104676,allnewfirehd8tablet8hddisplaywifi16gbincludesspecialoffersmagenta/5620406,allnewfirehd8tablet8hddisplaywifi16gbincludesspecialoffersmagenta/b01ahb9cn2",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",,TRUE,nan,0.0,5.0,"http://reviews.bestbuy.com/3545/5620406/reviews.htm?format=embedded&page=200,http://reviews.bestbuy.com/3545/5620406/reviews.htm?format=embedded&page=167",great for beginner or experienced person. Bought as a gift and she loves it,very fast,,,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Magenta",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Tablets,Tablets,Computers & Tablets","841667104676,amazon/53004484,amazon/b01ahb9cn2,0841667104676,allnewfirehd8tablet8hddisplaywifi16gbincludesspecialoffersmagenta/5620406,allnewfirehd8tablet8hddisplaywifi16gbincludesspecialoffersmagenta/b01ahb9cn2",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",,TRUE,nan,0.0,5.0,"http://reviews.bestbuy.com/3545/5620406/reviews.htm?format=embedded&page=200,http://reviews.bestbuy.com/3545/5620406/reviews.htm?format=embedded&page=167","Inexpensive tablet for him to use and learn on, step up from the NABI. He was thrilled with it, learn how to Skype on it already...",Beginner tablet for our 9 year old son.,,,DaveZ


In [8]:
def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x

In [9]:
target='reviews.rating'

In [10]:
X=get_independent_variables(reviews, target) 
print(X)

['reviews.id', 'reviews.numHelpful', 'id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer', 'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.sourceURLs', 'reviews.title', 'reviews.userCity', 'reviews.username', 'reviews.text', 'reviews.userProvince']


In [11]:
# Set target and predictor variables
y = target
y

'reviews.rating'

In [12]:
# Set up AutoML
aml = H2OAutoML(max_runtime_secs=run_time)

In [13]:
model_start_time = time.time()
aml.train(x=X,y=y,training_frame=reviews)

AutoML progress: |████████████████████████████████████████████████████████| 100%


H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Total input file size of  249  B is much larger than total cluster memory of Zero  , please use either a larger cluster or smaller data.
  Request: POST /3/Parse
    data: {'destination_frame': 'Key_Frame__upload_b1b15fa0966befbdf27c6d3f4bdf419f.hex', 'parse_type': 'CSV', 'separator': '44', 'single_quotes': 'False', 'check_header': '1', 'number_columns': '6', 'chunk_size': '4194304', 'delete_on_done': 'True', 'blocking': 'False', 'column_types': '["Numeric","String","Numeric","Numeric","Numeric","Numeric"]', 'column_names': '["","model_id","mean_residual_deviance","rmse","mae","rmsle"]', 'source_frames': '["upload_b1b15fa0966befbdf27c6d3f4bdf419f"]'}


In [ ]:
print(aml.leaderboard)

In [ ]:
best_model = h2o.get_model(aml.leaderboard[3,'model_id'])

In [ ]:
best_model.algo

In [ ]:
best_model_param = h2o.get_model(aml.leaderboard[0,'model_id'])

In [ ]:
best_model_param.params

In [ ]:
import pandas as pd

In [ ]:
reviews_=pd.read_csv("data/Amazon_review.csv")

In [ ]:
# define the properties missing value variable
properties_missing_df = reviews_.isnull().sum(axis=0).reset_index()
properties_missing_df.columns = ['column_name', 'missing_count']
properties_missing_df = properties_missing_df.ix[properties_missing_df['missing_count']>=0]
properties_missing_df = properties_missing_df.sort_values(by='missing_count')

# set the size and shape of the grapgh
ind = np.arange(properties_missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind, properties_missing_df.missing_count.values, color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(properties_missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")

# show the graph
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore", category = matplotlib.cbook.mplDeprecation)

In [ ]:
best_model.varimp_plot()